In [1]:
import pandas as pd
import numpy as np

In [2]:
DICT_COLNAMES = {'Fecha solicitud':'dateExam', 'Documento':'clientId','Prestacion':'nameExam', 'Resultado':'result', 'Orden':'requestId'}
BORRAR = ['Toma muestra','Fecha nacimiento','Validación','Procedencia','TipoDoc','Nombre', 'ApellidoPaterno','ApellidoMaterno','Edad','Sexo','Comuna','Dirección','Telefono','Email']

In [3]:
df_novus_bm = pd.read_excel('bm.xls', parse_dates=['Fecha solicitud'])

In [4]:
df_novus_bm = df_novus_bm.fillna('')

In [5]:
# cambiar de nombre columnas - usando dict_colnames
df_novus_bm = df_novus_bm.rename(columns=DICT_COLNAMES)

In [6]:
# formatear fecha '2022-08-01 08:46:05' a '2022-08-01'
df_novus_bm['dateExam'] = df_novus_bm['dateExam'].dt.date

In [7]:
#%%timeit
#df_novus_bm['clientName_2'] = df_novus_bm[['Nombre', 'ApellidoPaterno','ApellidoMaterno']].agg(' '.join, axis=1)
#6.55 * 10-3 segundos 

In [8]:
#%%timeit
df_novus_bm['clientName'] = df_novus_bm['Nombre'] + ' ' + df_novus_bm['ApellidoPaterno'] + ' ' + df_novus_bm['ApellidoMaterno']
#0.710 * 10-3 segundos

In [9]:
df_novus_bm = df_novus_bm.drop(BORRAR, axis = 1)

In [10]:
df_novus_bm

,dateExam,requestId,clientId,nameExam,result,clientName
0,2022-09-01,310100142,16937921-1,ANTIGENO,Negativo,CONSTANZA CAMILA CARRASCO SANCHEZ
1,2022-09-01,323100057,10176881-3,PCR SARS-CoV-2,Negativo,VIVIANA IDILIA INOSTROZA RIVAS
2,2022-09-01,333100002,21528598-7,PCR SARS-CoV-2,Negativo,JAIME IGNACIO PAZ OÑATE
3,2022-09-01,326100071,18622931-2,PCR SARS-CoV-2,Negativo,CAMILA VILLALON VALDEBENITO
4,2022-09-01,332100574,12664655-0,ANTIGENO,Negativo,MARCELO ANDRE VENEGAS LIZANA
...,...,...,...,...,...,...
177,2022-09-22,326100111,17405686-2,ANTIGENO,Negativo,JESSICA NICOLE ALVAREZ CAMPUSANO
178,2022-09-22,326100112,19671131-7,ANTIGENO,Negativo,SEBASTIAN LUIS FELIPE ALVAREZ CORNEJO
179,2022-09-22,322100004,22040496-K,PCR SARS-CoV-2,Negativo,BENJAMIN KIBLISKY SERMAN
180,2022-09-22,310100177,17412944-4,ANTIGENO,Negativo,PATRICIO ARCE ARAYA


In [11]:
#codeInternal	code	nameExam	nameIndicator	result	unitMeasurement	codeIndicator	category	outOfRange	categoryIndicator	referenceInf	referenceSup	examStatus	otherResults	requestStatus
df_novus_bm['unitMeasurement'] = np.nan
df_novus_bm['codeInternal'] = np.nan
df_novus_bm['code'] = np.nan
df_novus_bm['category'] = np.nan
df_novus_bm['nameIndicator'] = np.nan

# asignar nameIndicators
df_novus_bm['nameIndicator'][df_novus_bm['nameExam'].str.contains('ANTIGENO')] = 'CORONAVIRUS ANTI-SARS-CoV-2, IgG'
df_novus_bm['nameIndicator'][df_novus_bm['nameExam'].str.contains('PCR SARS-CoV-2|Test de saliva Covid-19')] = 'SARS-CoV-2 (COVID-19) RNA'


<ipython-input-11-1a5f84626ffe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novus_bm['nameIndicator'][df_novus_bm['nameExam'].str.contains('ANTIGENO')] = 'CORONAVIRUS ANTI-SARS-CoV-2, IgG'


In [12]:
loinc_db = pd.read_csv('loinc_db.csv', sep='\t', error_bad_lines=False)
loinc_db = loinc_db.dropna()

In [13]:
df_novus_bm = pd.merge(left=df_novus_bm, right=loinc_db, how='left', left_on=['nameExam', 'nameIndicator'], right_on=['Prestacion Orden', 'Prestación Estructura'])
df_novus_bm = df_novus_bm.drop(['Prestación Estructura', 'Prestacion Orden'], axis=1)

In [14]:
df_novus_bm['categoryIndicator'] = 'true-false'
df_novus_bm['outOfRange'] = np.where(df_novus_bm['result'].str.lower() == 'negativo', 'false', 'true')

In [17]:
df_novus_bm['referenceInf'] = np.nan
df_novus_bm['referenceSup'] = np.nan
df_novus_bm['otherResults'] = np.nan

# examStatus
df_novus_bm['examStatus'] = np.where(df_novus_bm['result'].isnull(), 'PENDING', 'COMPLETE')

#df_novus_bm['otherResults'] = np.where(df_novus_bm['categoryIndicator'] == 'other', df_novus_bm['Rango Ref'], np.nan)

# llenar resultados requestStatus. # si el resultado esta vacio, cambair requestStatus a Incomplete,
df_novus_bm['requestStatus'] = df_novus_bm['examStatus']


In [18]:
df_novus_bm

,dateExam,requestId,clientId,nameExam,result,clientName,unitMeasurement,codeInternal,code,category,nameIndicator,codeIndicator,categoryIndicator,outOfRange,referenceInf,referenceSup,otherResults,examStatus,requestStatus
0,2022-09-01,310100142,16937921-1,ANTIGENO,Negativo,CONSTANZA CAMILA CARRASCO SANCHEZ,NaN,NaN,NaN,NaN,"CORONAVIRUS ANTI-SARS-CoV-2, IgG",95209-3,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
1,2022-09-01,323100057,10176881-3,PCR SARS-CoV-2,Negativo,VIVIANA IDILIA INOSTROZA RIVAS,NaN,NaN,NaN,NaN,SARS-CoV-2 (COVID-19) RNA,94309-2,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
2,2022-09-01,333100002,21528598-7,PCR SARS-CoV-2,Negativo,JAIME IGNACIO PAZ OÑATE,NaN,NaN,NaN,NaN,SARS-CoV-2 (COVID-19) RNA,94309-2,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
3,2022-09-01,326100071,18622931-2,PCR SARS-CoV-2,Negativo,CAMILA VILLALON VALDEBENITO,NaN,NaN,NaN,NaN,SARS-CoV-2 (COVID-19) RNA,94309-2,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
4,2022-09-01,332100574,12664655-0,ANTIGENO,Negativo,MARCELO ANDRE VENEGAS LIZANA,NaN,NaN,NaN,NaN,"CORONAVIRUS ANTI-SARS-CoV-2, IgG",95209-3,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2022-09-22,326100111,17405686-2,ANTIGENO,Negativo,JESSICA NICOLE ALVAREZ CAMPUSANO,NaN,NaN,NaN,NaN,"CORONAVIRUS ANTI-SARS-CoV-2, IgG",95209-3,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
178,2022-09-22,326100112,19671131-7,ANTIGENO,Negativo,SEBASTIAN LUIS FELIPE ALVAREZ CORNEJO,NaN,NaN,NaN,NaN,"CORONAVIRUS ANTI-SARS-CoV-2, IgG",95209-3,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
179,2022-09-22,322100004,22040496-K,PCR SARS-CoV-2,Negativo,BENJAMIN KIBLISKY SERMAN,NaN,NaN,NaN,NaN,SARS-CoV-2 (COVID-19) RNA,94309-2,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
180,2022-09-22,310100177,17412944-4,ANTIGENO,Negativo,PATRICIO ARCE ARAYA,NaN,NaN,NaN,NaN,"CORONAVIRUS ANTI-SARS-CoV-2, IgG",95209-3,true-false,FALSE,NaN,NaN,NaN,COMPLETE,COMPLETE
